## **Utilisation simple des identifieurs de langues** (rev1) **avec les :**
>## **- Tokenisations BERT ou Tiktoken**
>## **- Classificateurs Naïve Bayes et Gradiant Boosting**

#### **Choix du tokenizer**

In [1]:
import pandas as pd
import random
import joblib
from sklearn.metrics import accuracy_score, classification_report

# Choix de la Tokenisation (False = BERT, True Tiktoken)
titoken_tokenization = True

## Pour résoudre les problème de mémoire et de performances
nb_phrase_lang = 1000000

import warnings
warnings.filterwarnings('ignore')

#### **Lectures des phrases de "sentences.csv", et de leur étiquette "Langue" pour les langues sélectionnées**

In [2]:
# Ouvrir le fichier d'entrée en mode lecture
def create_lang_df(path):
    df = pd.read_csv(path, index_col ='id')
    return df

df_big = create_lang_df('../data/multilingue/sentences.csv')
lan_code = ['eng','fra','deu','spa','ita']
df = pd.DataFrame(columns=df_big.columns)
for i in range(len(lan_code)):
    df= pd.concat([df, df_big[df_big['lan_code']==lan_code[i]].iloc[:nb_phrase_lang]])
df = df.sample(frac=1, random_state=3).reset_index(drop=True)
n_rows = len(df)
print('Nombre de lignes de sentence.csv:',n_rows)
nb_phrases_lang =[]
for l in lan_code:
    nb_phrases_lang.append(sum(df['lan_code']==l))
print("Nombre de phrases par langue ",lan_code,":",nb_phrases_lang)
display(df.head(10))
display(df.tail(10))

Nombre de lignes de sentence.csv: 1750000
Nombre de phrases par langue  ['eng', 'fra', 'deu', 'spa', 'ita'] : [350000, 350000, 350000, 350000, 350000]


,lan_code,sentence
0,eng,She is afraid of death.
1,ita,Indovina cosa scelgo io.
2,spa,"¿Puedo ayudarlo? ""No, gracias. Solo estoy mira..."
3,ita,Io non sono una fricchettona!
4,deu,"Es sind schon fast 10 Jahre vergangen, aber du..."
5,spa,Creía que me quería.
6,eng,This school sets high moral standards for pupils.
7,eng,"Man is judged by his courage, woman by her charm."
8,fra,Je mange des pruneaux sucrés.
9,fra,J'ai écrit une chanson pour toi.


,lan_code,sentence
1749990,deu,Es geschieht heutzutage ja so viel in unserer ...
1749991,spa,El almuerzo está preparado.
1749992,eng,I've seen enough.
1749993,ita,Hanno accelerato il passo.
1749994,fra,Elle en pince pour ce garçon.
1749995,deu,"Wer von uns wünschte nicht manchmal, dass er d..."
1749996,ita,No! Io odio i broccoli!
1749997,fra,Tu seras tuée !
1749998,fra,Tom aurait dû manger plus.
1749999,eng,He took the video to a local TV station.


#### **Selection du tokenizer** en fonction de la variable titoken_tokenization

In [3]:
# Selection du tokenizer
if titoken_tokenization:
    import tiktoken
    tokenizer = tiktoken.get_encoding("cl100k_base")
else:
    from transformers import BertTokenizerFast
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-uncased')

#### **Chargement des Tokens utilisés**

In [4]:
from collections import Counter
import pickle

def load_dict_token():
    if titoken_tokenization: path = '../data/dict_token_tiktoken'
    else: path = '../data/dict_token_BERT'
    
    with open(path, 'rb') as fichier:
        dict_ids = pickle.load(fichier)
        # Définition d'une liste 'écrite' des tokens
        decoded_keys = [tokenizer.decode([key]) for key in list(dict_ids.keys())]
    return dict_ids, decoded_keys

#### **Choix du nom du fichier de sauvegarde du classifieur**

In [5]:
def get_file_name(titoken_tokenization, classifier):
    if titoken_tokenization:
        return "id_lang_tiktoken_"+classifier+".pkl"
    else:
        return "id_lang_BERT_"+classifier+".pkl"

#### **Création d'un classificateur avec l'algorithme Naïve Bayes**

In [6]:
from sklearn import naive_bayes

# Chargement du classificateur sauvé
clf_nb = joblib.load("../data/"+get_file_name(titoken_tokenization,"nb"))
dict_ids, decoded_keys = load_dict_token()


#### **Création d'un classificateur avec l'algorithme Gradiant Boosting**

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

# Chargement du classificateur sauvé
clf_gb = joblib.load("../data/"+get_file_name(titoken_tokenization,"gb"))
######### dict_ids, decoded_keys = load_dict_token() ######### 

#### **Definition de fonctions identificateur de langue**

In [8]:
import numpy as np

# Créez un DataFrame BOW avec les phrases (lignes) et les fréquences de chaque token (colonnes)
def create_BOW(data):
    BOW = []
    try:
        if 'str' in str(type(data)):
            l_tokenised = tokenizer.encode(data)
            BOW.append([l_tokenised.count(token) for token in dict_ids])
        else:
            for ligne in data:
                l_tokenised = tokenizer.encode(ligne)
                BOW.append([l_tokenised.count(token) for token in dict_ids])
    except:
        BOW.append([tokenizer.encode(" ").count(token) for token in dict_ids])
    return BOW

def lang_id_nb(sentences):
    return clf_nb.predict(create_BOW(sentences))

def lang_id_gb(sentences):
    return clf_gb.predict(create_BOW(sentences))

#### **Exemples d'utilisation**

In [9]:
import random
# Instanciation d'exemples

sentence_no = random.sample(range(len(df)),5)

exemples = ["france is often snowy during spring , and it is relaxing in january .",
           "elle adore les voitures très luxueuses, et toi ?",
           "she loves very luxurious cars, don't you?",
           "vamos a la playa",
           "Ich heiße Keyne, und das ist wunderbar",
           "she loves you, mais elle te hait aussi, and das ist traurig", # Attention à cette phrase trilingue
           "I ate caviar", 
           "Vogliamo visitare il Colosseo e nuotare nel Tevere.",
            df['sentence'].iloc[sentence_no[0]],
            df['sentence'].iloc[sentence_no[1]],
            df['sentence'].iloc[sentence_no[2]],
            df['sentence'].iloc[sentence_no[3]],
            df['sentence'].iloc[sentence_no[4]],
          ]
lang_exemples = ['eng','fra','eng','spa','deu','e,f,d','en','ita',df['lan_code'].iloc[sentence_no[0]],df['lan_code'].iloc[sentence_no[1]],df['lan_code'].iloc[sentence_no[2]],
                 df['lan_code'].iloc[sentence_no[3]],df['lan_code'].iloc[sentence_no[4]]]
print('no\t lang\t Phrase')                            
for i in range(len(exemples)):
    print(i,'-\t',lang_exemples[i],'\t',exemples[i])

no	 lang	 Phrase
0 -	 eng 	 france is often snowy during spring , and it is relaxing in january .
1 -	 fra 	 elle adore les voitures très luxueuses, et toi ?
2 -	 eng 	 she loves very luxurious cars, don't you?
3 -	 spa 	 vamos a la playa
4 -	 deu 	 Ich heiße Keyne, und das ist wunderbar
5 -	 e,f,d 	 she loves you, mais elle te hait aussi, and das ist traurig
6 -	 en 	 I ate caviar
7 -	 ita 	 Vogliamo visitare il Colosseo e nuotare nel Tevere.
8 -	 ita 	 Tom è impegnato adesso, per cui non può parlare con te.
9 -	 ita 	 Faccio colazione alle sette ogni mattina.
10 -	 ita 	 Fuori ci sono i bambini che giocano e mettono allegria.
11 -	 fra 	 L'année dernière je suis allé en Belgique et en Israël.
12 -	 fra 	 Peu reste à faire.


In [10]:
# Affichage des prédictions
print("Langue réelle\tPréd. Naive B.\tPréd. Grad. B.\tPhrase")
for i in range(len(exemples)):
    print(lang_exemples[i]+'\t\t'+lang_id_nb(exemples[i])[0]+'\t\t'+lang_id_gb(exemples[i])[0]+'\t\t'+exemples[i])


Langue réelle	Préd. Naive B.	Préd. Grad. B.	Phrase
eng		eng		eng		france is often snowy during spring , and it is relaxing in january .
fra		fra		fra		elle adore les voitures très luxueuses, et toi ?
eng		eng		eng		she loves very luxurious cars, don't you?
spa		spa		spa		vamos a la playa
deu		deu		deu		Ich heiße Keyne, und das ist wunderbar
e,f,d		fra		deu		she loves you, mais elle te hait aussi, and das ist traurig
en		ita		eng		I ate caviar
ita		ita		ita		Vogliamo visitare il Colosseo e nuotare nel Tevere.
ita		ita		ita		Tom è impegnato adesso, per cui non può parlare con te.
ita		ita		ita		Faccio colazione alle sette ogni mattina.
ita		ita		ita		Fuori ci sono i bambini che giocano e mettono allegria.
fra		fra		fra		L'année dernière je suis allé en Belgique et en Israël.
fra		fra		fra		Peu reste à faire.


> **Recherche des phrases mal classées par Naive Bayes**

In [11]:
n_bad_max = 30
n_bad = 0
for i in range(len(df)):
    if (n_bad<n_bad_max):
        if (df['lan_code'].iloc[i] != lang_id_nb(df['sentence'].iloc[i])):
            n_bad +=1
            print(n_bad,'- No',i,' - Réel:',df['lan_code'].iloc[i],' Prédit:',lang_id_nb(df['sentence'].iloc[i])[0],'    ',
                  df['sentence'].iloc[i]," (proba={:.2f}".format(max(clf_nb.predict_proba(np.array(create_BOW([df['sentence'].iloc[i]])))[0]))+")")


1 - No 116  - Réel: eng  Prédit: spa      A guilty conscience needs no accuser.  (proba=0.70)
2 - No 185  - Réel: ita  Prédit: spa      Me lo traduci?  (proba=0.97)
3 - No 209  - Réel: spa  Prédit: ita      Admiro tu talento.  (proba=0.60)
4 - No 725  - Réel: fra  Prédit: spa      Tom a changé de boulot.  (proba=0.59)
5 - No 731  - Réel: fra  Prédit: spa      Mary a aidé Tom.  (proba=0.89)
6 - No 921  - Réel: ita  Prédit: fra      Ce la caveremo.  (proba=0.97)
7 - No 1155  - Réel: spa  Prédit: ita      Usted miente cual periódico.  (proba=0.55)
8 - No 1210  - Réel: ita  Prédit: spa      Tutto sarà nuovo?  (proba=0.58)
9 - No 1254  - Réel: spa  Prédit: ita      La sala necesita una cortina nueva.  (proba=0.61)
10 - No 1297  - Réel: eng  Prédit: spa      No dirty jokes!  (proba=0.67)
11 - No 1325  - Réel: deu  Prédit: spa      Tom beleidigte Mary.  (proba=0.34)
12 - No 1743  - Réel: ita  Prédit: spa      Marie era esigente.  (proba=0.88)
13 - No 1841  - Réel: ita  Prédit: spa      Lo odi